# Basisanalyse

## Benötigte Bibliotheken und Module

In [2]:
# Benötigte Bibliotheken und Module
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from geopy.distance import geodesic
import folium
from shapely import wkt
import seaborn as sns
import requests
from branca.element import Template, MacroElement
from branca.element import Element, Html
import re
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import contextily as ctx


## Einlesen der CSV-Dateien

In [3]:
# CSV-Datei einlesen
file_path_persons = '../../data/clean/Data_Masterarbeit - list_named_after_humans.csv'
file_path_all_streets = '../../data/clean/Data_Masterarbeit - list_complete.csv'
file_path_street_layout = "../../data/raw/lhd_strassenzuege.csv"

df_persons = pd.read_csv(file_path_persons, sep=",")
df_all = pd.read_csv(file_path_all_streets, sep=",")
df_streets = pd.read_csv(file_path_street_layout, sep=";")

## Basisanalyse

In [4]:
# Grundwort, Benennungstyp und lokaler Bezug

# Schritt 1: Kategorien bestimmen
categories = ["Grundwort", "Benennungstyp", "lokaler Bezug"]

# Kategorienschleife
for category in categories:
    # Schritt 2: Kategorien zählen 
    category_counter = df_all[category].value_counts()
    total = category_counter.sum()

    #nur Werte mit > 10 Einträgen
    category_filtered = category_counter[category_counter >= 10]

    # Schritt 3: Visualisierung
    data_for_plot = category_filtered.iloc[::-1]

    farben = plt.cm.tab20.colors
    
    plt.figure(figsize=(10, 8))

    # Diagrammaufruf
    bars = plt.barh(
        data_for_plot.index,    # y-Achse (Labels)
        data_for_plot.values,   # x-Achse (Werte/Breite)
        color=farben[:len(data_for_plot)]
    )

    labels_for_bars = []
    for value in data_for_plot.values:
        pct = (value / total) * 100
        # WICHTIG: 'value' IST bereits der absolute Wert!
        labels_for_bars.append(f" {pct:.1f}% ({value})") # Leerzeichen für Abstand

    plt.bar_label(
        bars,
        labels=labels_for_bars,
        padding=5,  # Abstand vom Balken
        fontsize=11
    )

    plt.title(f"Verteilung nach {category}")
    plt.xlabel("Anzahl") # Achsenbeschriftung hinzufügen
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    
    # X-Achsen-Limit anpassen, damit Labels Platz haben
    plt.xlim(0, data_for_plot.values.max() * 1.18)

    plt.tight_layout() 
    
    #Diagramm speichern
    plt.savefig(f"../../results/charts/bar_charts/Basisanalyse_{category}_Balkendiagramm.png", dpi=300)
    plt.close()

### Distanzanalyse

In [ ]:
# WKT in Geometrie umwandeln
def safe_wkt(val):
    if pd.isna(val) or str(val).strip() == "":
        return None
    try:
        return wkt.loads(str(val).replace("Point", "POINT"))
    except:
        return None

df_all["geometry"] = df_all["Koordinaten [WD]"].apply(safe_wkt)
df_all["lon"] = df_all["geometry"].apply(lambda g: g.x if g else np.nan)
df_all["lat"] = df_all["geometry"].apply(lambda g: g.y if g else np.nan)

# Merge über ID
df = df_all.copy()

# Altmarkt Dresden als Zentrum
center = (51.0504, 13.7373)

# Distanz berechnen
print("Berechne 'distanz_km'...") # Statusmeldung
df["distanz_km"] = np.nan
mask = df["lat"].notna() & df["lon"].notna()
df.loc[mask, "distanz_km"] = df.loc[mask].apply(
    lambda row: geodesic((row["lat"], row["lon"]), center).kilometers,
    axis=1
)

# Wähle die Spalte aus, die analysiert werden soll
kategorie_spalte = "Grundwort" 

# Daten filtern (NaNs in Distanz ODER Kategorie entfernen)
plot_data = df[df["distanz_km"].notna() & df[kategorie_spalte].notna()]

# Finde die Top 15 Kategorien (damit der Plot lesbar bleibt)
top_kategorien = plot_data[kategorie_spalte].value_counts().nlargest(15).index

# Filtere die Daten auf diese Top-Kategorien
plot_data = plot_data[plot_data[kategorie_spalte].isin(top_kategorien)]

# Reihenfolge der Kategorien (nach Median-Distanz sortiert)
order = plot_data.groupby(kategorie_spalte)["distanz_km"] \
                 .median() \
                 .sort_values() \
                 .index

# Statistische Kennzahlen je Kategorie
stats = plot_data.groupby(kategorie_spalte)["distanz_km"] \
                 .agg(["mean", "median", "count"]) \
                 .reindex(order) # .reindex() stellt sicher, dass 'stats' dieselbe Reihenfolge hat wie 'order'

# Boxplot zeichnen
plt.figure(figsize=(12, 10)) 
sns.boxplot(
    data=plot_data,       
    x=kategorie_spalte,   
    y="distanz_km",
    order=order           
)
plt.title(f"Distanz der Straßen nach 'Grundwort' zum Altmarkt (Dresden)")
plt.xlabel("Grundwort")
plt.ylabel("Distanz zum Zentrum (km)")

# Dreht die x-Achsen-Beschriftungen, damit sie lesbar sind
plt.xticks(rotation=75, ha='right', fontsize=12)

# Mittelwert & Median über jeder Box eintragen
for i, (kategorie, row) in enumerate(stats.iterrows()):
    mean_val = row["mean"]
    median_val = row["median"]
    plt.text(
        i,
        mean_val + 0.2, # Position leicht über dem Mittelwert
        f"Ø {mean_val:.2f}\nMed {median_val:.2f}",
        ha="center",
        color="black",
        fontsize=12,
        bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=0.1)
    )

# Legende für Erklärung
plt.legend(
    [plt.Line2D([0], [0], color="white")],
    ["Ø = Mittelwert | Med = Median"],
    loc="upper left",
    frameon=False
)


plt.tight_layout() 

# Diagramm speichern
plt.savefig("../../results/charts/boxplots/distanzanalyse_Grundwort_boxplot.png", dpi=300)
plt.close()

Berechne 'distanz_km'...


## Geografische Verteilung

### Stadtbezirke

#### Folium Karten

In [ ]:
# 1. Lokale Straßendaten laden (df_streets)

# WKT-Geometrien bereinigen und umwandeln
def parse_wkt_safe(x):
    if pd.isna(x):
        return None
    try:
        return wkt.loads(str(x))
    except:
        return None

df_streets["geom_clean"] = df_streets["geom"].astype(str).str.replace(r"SRID=\d+;", "", regex=True)
df_streets["geometry"] = df_streets["geom_clean"].apply(parse_wkt_safe)
strassen_gdf = gpd.GeoDataFrame(df_streets.dropna(subset=["geometry"]), geometry="geometry", crs="EPSG:4326")

# Straßennamen vereinheitlichen
strassen_gdf["strasse_key"] = strassen_gdf["strasse"].str.strip().str.lower()

# 2. Kategorie-Daten laden (df_all)

df_all["strasse_key"] = df_all["Straßenname"].str.strip().str.lower()

# Merge: Straßen + Kategorie
strassen_merge = strassen_gdf.merge(
    df_all[["strasse_key", "Benennungstyp"]], # Nur die benötigten Spalten aus df_all
    left_on="strasse_key",
    right_on="strasse_key",
    how="left"
)


# 3. Stadtbezirke laden (GeoJSON)
stadtbezirke_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L139/items"
try:
    stadt_json = requests.get(stadtbezirke_url).json()
    stadt_gdf = gpd.GeoDataFrame.from_features(stadt_json["features"])
    stadt_gdf = stadt_gdf.set_crs(epsg=4326)
except Exception as e:
    print(f"Fehler beim Laden der Stadtbezirke: {e}")
    # Hier ggf. Skript beenden oder mit lokalen Daten weitermachen

# 4. Räumliche Verknüpfung Straße ↔ Bezirk
# WICHTIG: Sicherstellen, dass beide GeoDataFrames ein gültiges CRS haben
if strassen_gdf.crs != stadt_gdf.crs:
    print("Warnung: CRS stimmen nicht überein. Projiziere strassen_gdf neu...")
    strassen_gdf = strassen_gdf.to_crs(stadt_gdf.crs)

strassen_bezirk = gpd.sjoin(strassen_merge, stadt_gdf, how="left", predicate="intersects")

# VORBEREITUNG FÜR DIE SCHLEIFE

# Alle einzigartigen Benennungstypen finden
all_types = strassen_bezirk["Benennungstyp"].dropna().unique()

# Gesamtanzahl aller nach Personen/Typen benannter Straßen pro Bezirk (Nenner für die Prozentrechnung)
personen_strassen = strassen_bezirk.dropna(subset=["Benennungstyp"])
personen_strassen_unique = personen_strassen.drop_duplicates(subset=["bez", "strasse_key"])
gesamt_pro_bezirk = personen_strassen_unique.groupby("bez").size()

# Kartenmittelpunkt (muss auch nur einmal berechnet werden)
center = stadt_gdf.geometry.centroid.unary_union.centroid


print("Starte interaktive Kartenerstellung (für Screenshots, ohne LayerControl)...")

# START: Iteration durch alle Benennungstypen
for ausgewaehltes_feld in all_types:
    
    print(f"\n--- Verarbeite Kategorie: {ausgewaehltes_feld} ---")

    # 5. Filtern
    strassen_feld = strassen_bezirk[strassen_bezirk["Benennungstyp"] == ausgewaehltes_feld]
    strassen_feld_unique = strassen_feld.drop_duplicates(subset=["bez", "strasse_key"])

    # 6. Prozentrechnung
    feld_counts = strassen_feld_unique.groupby("bez").size()
    anteil_feld = (feld_counts / gesamt_pro_bezirk).fillna(0).reset_index()
    anteil_feld.columns = ["bez", "anteil_feld"]
    stadt_gdf_feld = stadt_gdf.merge(anteil_feld, on="bez", how="left")
    stadt_gdf_feld["anteil_feld"] = stadt_gdf_feld["anteil_feld"].fillna(0)

    # Formatierte Spalte für den Tooltip
    stadt_gdf_feld["anteil_text"] = (stadt_gdf_feld["anteil_feld"] * 100).round(1).astype(str) + " %"

    # 7. Karte erstellen
    karte = folium.Map(location=[center.y, center.x], zoom_start=12)

    # Choropleth für Bezirke (direkt zu 'karte' hinzufügen)
    folium.Choropleth(
        geo_data=stadt_gdf_feld,
        data=stadt_gdf_feld,
        columns=["bez", "anteil_feld"],
        key_on="feature.properties.bez",
        fill_color="PuBu",
        fill_opacity=0.6, # Passen Sie die Transparenz nach Bedarf an
        line_opacity=0.2,
        legend_name=f"Anteil '{ausgewaehltes_feld}' Straßenbenennungen pro Bezirk"
    ).add_to(karte) # <-- Direkt zu 'karte'

    # Tooltip (direkt zu 'karte' hinzufügen)
    folium.GeoJson(
        stadt_gdf_feld,
        tooltip=folium.GeoJsonTooltip(
            fields=["bez", "anteil_text"],
            aliases=["Bezirk", f"Anteil: "],
            localize=True,
            sticky=False
        ),
        style_function=lambda x: {"fillOpacity": 0, "color": "black", "weight": 0.5}
    ).add_to(karte) # <-- Direkt zu 'karte'

    # 8. Straßen der Kategorie einzeichnen
    farbe_strassen = "red"
    for _, row in strassen_feld.iterrows():
        geom = row['geometry']
        if geom.geom_type == "LineString":
            punkte = [(pt[1], pt[0]) for pt in geom.coords]
            folium.PolyLine(punkte, color=farbe_strassen, weight=2, opacity=0.7).add_to(karte) # <-- Direkt zu 'karte'
        elif geom.geom_type == "MultiLineString":
            for line in geom.geoms:
                punkte = [(pt[1], pt[0]) for pt in line.coords]
                folium.PolyLine(punkte, color=farbe_strassen, weight=2, opacity=0.7).add_to(karte) # <-- Direkt zu 'karte'

    # Überschrift zur Karte hinzufügen
    title_text = f"Verteilung der Straßenbenennungen: '{ausgewaehltes_feld}'"
    title_html = f"""
                 <h3 align="center" style="font-size:16px; font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;">
                 <b>{title_text}</b>
                 </h3>
                 """
    karte.get_root().html.add_child(Element(title_html))

    # 9. Karte speichern
    safe_filename = re.sub(r'[^\w\-_\. ]', '_', ausgewaehltes_feld)
    filename = f"../../results/maps/Stadtbezirke/Karte_Stadtbezirk_Benennungstyp_{safe_filename}.html"
    
    karte.save(filename)

print("\n--- Alle interaktiven Folium-Karten erfolgreich erstellt. ---")

C:\Users\juliu\AppData\Local\Temp\ipykernel_2376\3823615607.py:61: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = stadt_gdf.geometry.centroid.unary_union.centroid
C:\Users\juliu\AppData\Local\Temp\ipykernel_2376\3823615607.py:61: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = stadt_gdf.geometry.centroid.unary_union.centroid


Starte interaktive Kartenerstellung (für Screenshots, ohne LayerControl)...

--- Verarbeite Kategorie: Person real ---

--- Verarbeite Kategorie: Ort ---

--- Verarbeite Kategorie: Region ---

--- Verarbeite Kategorie: Bevölkerungsgruppe ---

--- Verarbeite Kategorie: Bauwerk/Infrastruktur ---

--- Verarbeite Kategorie: Sonstige ---

--- Verarbeite Kategorie: Selbstreferenziell ---

--- Verarbeite Kategorie: Natur ---

--- Verarbeite Kategorie: Gegenstand ---

--- Verarbeite Kategorie: weitere Geografika ---

--- Verarbeite Kategorie: Gewerbe/Industrie ---

--- Verarbeite Kategorie: Gruppe real ---

--- Verarbeite Kategorie: Beruf/Funktion ---

--- Verarbeite Kategorie: Abstrakta ---

--- Verarbeite Kategorie: Ereignis ---

--- Verarbeite Kategorie: Person fiktiv ---

--- Verarbeite Kategorie: Unbekannt ---

--- Verarbeite Kategorie: Siedlung ---

--- Verarbeite Kategorie: Burg ---

--- Verarbeite Kategorie: Land ---

--- Verarbeite Kategorie: Kunstwerk ---

--- Alle interaktiven Foliu

#### PNG's

In [15]:
# 1. Lokale Straßendaten laden (df_streets)

# WKT-Geometrien bereinigen und umwandeln
def parse_wkt_safe(x):
    if pd.isna(x):
        return None
    try:
        return wkt.loads(str(x))
    except:
        return None

df_streets["geom_clean"] = df_streets["geom"].astype(str).str.replace(r"SRID=\d+;", "", regex=True)
df_streets["geometry"] = df_streets["geom_clean"].apply(parse_wkt_safe)
strassen_gdf = gpd.GeoDataFrame(df_streets.dropna(subset=["geometry"]), geometry="geometry", crs="EPSG:4326")

# Straßennamen vereinheitlichen
strassen_gdf["strasse_key"] = strassen_gdf["strasse"].str.strip().str.lower()

# 2. Kategorie-Daten laden (df_all)

df_all["strasse_key"] = df_all["Straßenname"].str.strip().str.lower()

# Merge: Straßen + Kategorie
strassen_merge = strassen_gdf.merge(
    df_all[["strasse_key", "Benennungstyp"]], # Nur die benötigten Spalten aus df_all
    left_on="strasse_key",
    right_on="strasse_key",
    how="left"
)


# 3. Stadtbezirke laden (GeoJSON)
stadtbezirke_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L139/items"
try:
    stadt_json = requests.get(stadtbezirke_url).json()
    stadt_gdf = gpd.GeoDataFrame.from_features(stadt_json["features"])
    stadt_gdf = stadt_gdf.set_crs(epsg=4326)
except Exception as e:
    print(f"Fehler beim Laden der Stadtbezirke: {e}")
    # Hier ggf. Skript beenden oder mit lokalen Daten weitermachen

# 4. Räumliche Verknüpfung Straße ↔ Bezirk
if strassen_gdf.crs != stadt_gdf.crs:
    print("Warnung: CRS stimmen nicht überein. Projiziere strassen_gdf neu...")
    strassen_gdf = strassen_gdf.to_crs(stadt_gdf.crs)

strassen_bezirk = gpd.sjoin(strassen_merge, stadt_gdf, how="left", predicate="intersects")

# VORBEREITUNG FÜR DIE SCHLEIFE

# Alle einzigartigen Benennungstypen finden
all_types = strassen_bezirk["Benennungstyp"].dropna().unique()

# Gesamtanzahl aller nach Personen/Typen benannter Straßen pro Bezirk (Nenner für die Prozentrechnung)
personen_strassen = strassen_bezirk.dropna(subset=["Benennungstyp"])
personen_strassen_unique = personen_strassen.drop_duplicates(subset=["bez", "strasse_key"])
gesamt_pro_bezirk = personen_strassen_unique.groupby("bez").size()

# Kartenmittelpunkt
center = stadt_gdf.geometry.centroid.unary_union.centroid


print("Starte Erstellung von statischen PNG-Karten...")

# START: Iteration durch alle Benennungstypen
for ausgewaehltes_feld in all_types:
    
    print(f"\n--- Verarbeite Kategorie: {ausgewaehltes_feld} ---")

    # 5. Filtern
    strassen_feld = strassen_bezirk[strassen_bezirk["Benennungstyp"] == ausgewaehltes_feld]

    # 6. Prozentrechnung
    strassen_feld_unique = strassen_feld.drop_duplicates(subset=["bez", "strasse_key"])
    feld_counts = strassen_feld_unique.groupby("bez").size()
    anteil_feld = (feld_counts / gesamt_pro_bezirk).fillna(0).reset_index()
    anteil_feld.columns = ["bez", "anteil_feld"]
    stadt_gdf_feld = stadt_gdf.merge(anteil_feld, on="bez", how="left")
    stadt_gdf_feld["anteil_feld"] = stadt_gdf_feld["anteil_feld"].fillna(0)

    # Spalte für die Legende (Werte von 0-100)
    stadt_gdf_feld["anteil_prozent"] = stadt_gdf_feld["anteil_feld"] * 100

    
    # 7. Daten für Plotting vorbereiten (CRS umwandeln)
    #    Basemaps benötigen EPSG:3857 (Web Mercator)
    target_crs = "EPSG:3857"
    stadt_plot = stadt_gdf_feld.to_crs(target_crs)
    strassen_plot = strassen_feld.to_crs(target_crs)

    # 8. Karte erstellen
    fig, ax = plt.subplots(figsize=(12, 12)) # 12x12 Zoll für gute Auflösung

    # A: Bezirke als Choropleth plotten
    stadt_plot.plot(
        ax=ax,
        column="anteil_prozent", # Spalte für die Farbe
        cmap="PuBu",          # Farbschema
        legend=True,          # Zeigt die Legende
        alpha=0.6,
        edgecolor='black',      
        linewidth=0.5,           
        legend_kwds={
            'label': f"Anteil '{ausgewaehltes_feld}' (%)",
            'shrink': 0.75, 
            'aspect': 30   
        }
    )
    
    # Legenden-Schriftgröße anpassen
    cax = fig.axes[-1] 
    cax.tick_params(labelsize=8) 
    cax.yaxis.label.set_size(9)  

    # B: Straßen-Linien plotten
    strassen_plot.plot(
        ax=ax,
        color="red",
        linewidth=0.7,      
        alpha=0.5           
    )

    # C: Hintergrundkarte hinzufügen
    ctx.add_basemap(
        ax,
        crs=target_crs,
        source=ctx.providers.OpenStreetMap.Mapnik,
        zoom=13
    )

    # 9. Karte finalisieren & als PNG speichern
    ax.set_axis_off() 
    plt.title(f"Verteilung Straßennamen: {ausgewaehltes_feld}", fontsize=16)
    
    # Dateinamen "säubern"
    safe_filename = re.sub(r'[^\w\-_\. ]', '_', ausgewaehltes_feld)
    
    # Ordner für PNGs
    filename = f"../../results/images/Stadtbezirke/Karte_Stadtbezirk_Benennungstyp_{safe_filename}.png"
    
    plt.savefig(
        filename,
        dpi=400, 
        bbox_inches="tight" 
    )
    plt.close(fig) 

print("\n--- Alle statischen PNG-Karten erfolgreich erstellt. ---")

C:\Users\juliu\AppData\Local\Temp\ipykernel_19916\1448827935.py:60: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = stadt_gdf.geometry.centroid.unary_union.centroid
C:\Users\juliu\AppData\Local\Temp\ipykernel_19916\1448827935.py:60: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = stadt_gdf.geometry.centroid.unary_union.centroid


Starte Erstellung von statischen PNG-Karten...

--- Verarbeite Kategorie: Person real ---

--- Verarbeite Kategorie: Ort ---

--- Verarbeite Kategorie: Region ---

--- Verarbeite Kategorie: Bevölkerungsgruppe ---

--- Verarbeite Kategorie: Bauwerk/Infrastruktur ---

--- Verarbeite Kategorie: Sonstige ---

--- Verarbeite Kategorie: Selbstreferenziell ---

--- Verarbeite Kategorie: Natur ---

--- Verarbeite Kategorie: Gegenstand ---

--- Verarbeite Kategorie: weitere Geografika ---

--- Verarbeite Kategorie: Gewerbe/Industrie ---

--- Verarbeite Kategorie: Gruppe real ---

--- Verarbeite Kategorie: Beruf/Funktion ---

--- Verarbeite Kategorie: Abstrakta ---

--- Verarbeite Kategorie: Ereignis ---

--- Verarbeite Kategorie: Person fiktiv ---

--- Verarbeite Kategorie: Unbekannt ---

--- Verarbeite Kategorie: Siedlung ---

--- Verarbeite Kategorie: Burg ---

--- Verarbeite Kategorie: Land ---

--- Verarbeite Kategorie: Kunstwerk ---

--- Alle statischen PNG-Karten erfolgreich erstellt. ---

### Stadtbezirksübersicht

In [10]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# 1. Absolute Anzahl pro Bezirk & Tätigkeitsfeld
counts = personen_strassen_unique.groupby(["bez", "Benennungstyp"]).size().unstack(fill_value=0)

# Prozentualer Anteil
anteile = counts.div(counts.sum(axis=1), axis=0)

# 2. Konsistente Farben

# 2a. Repliziere die Logik aus Block 1, um die "Haupt-Typen" zu finden
b1_counter = df_all["Benennungstyp"].value_counts()
b1_filtered = b1_counter[b1_counter >= 10]
# Dies ist die geordnete Liste, die in Block 1 die Farben bestimmt:
b1_ordered_names = b1_filtered.iloc[::-1].index 

# 2b. Hole den Farb-Pool
farben_pool = cm.tab20.colors 

# 2c. Erstelle das Farb-Mapping für die Haupt-Typen
farbe_map = {}
for i, name in enumerate(b1_ordered_names):
    farbe_map[name] = mcolors.to_hex(farben_pool[i % len(farben_pool)])

# 2d. Erweitere das Mapping für Typen, die in Block 1 < 10 waren
# (Wir verwenden 'lightgrey' für diese, um sie klar abzugrenzen)
all_types_in_chart = anteile.columns
for name in all_types_in_chart:
    if name not in farbe_map:
        farbe_map[name] = 'lightgrey' # Farbe für "Sonstige/Rest"

# 3. Gestapeltes Balkendiagramm

fig, ax = plt.subplots(figsize=(10, 8)) # Größe angepasst für H-Layout

# 3a. Diagrammaufruf
anteile.plot(
    kind="barh", 
    stacked=True, 
    color=[farbe_map.get(f, 'grey') for f in anteile.columns],
    ax=ax
)

# 3b. Achsenbeschriftungen getauscht
ax.set_xlabel("Anteil der Straßen")
ax.set_ylabel("Bezirk")
ax.set_title("Prozentuale Anteile der Benennungstypen pro Bezirk")

# 3c. X-Achse auf Prozent formatieren
ax.set_xticklabels([f'{int(x*100)}%' for x in ax.get_xticks()])
ax.set_xlim(0, 1)


# 4. Absolute Zahl

for i, bezirk in enumerate(counts.index):
    total = counts.loc[bezirk].sum()
    ax.text(1.01, i, str(total), ha='left', va='center', fontweight='bold')

# 5. Legende und Layout
ax.invert_yaxis()
plt.legend(title="Benennungstyp", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()

# 6. Diagramm speichern
plt.savefig(f"../../results/charts/bar_charts_district_comparison/Balkendiagramm_prozentuale_Anteile_Stadtbezirke_horizontal.png", dpi=300)
plt.close()

C:\Users\juliu\AppData\Local\Temp\ipykernel_19916\2898390959.py:56: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([f'{int(x*100)}%' for x in ax.get_xticks()])


### Gemarkungen

#### Folium Karten

In [ ]:
# 1. Lokale Straßendaten laden (df_streets)

# WKT-Geometrien bereinigen und umwandeln
def parse_wkt_safe(x):
    if pd.isna(x):
        return None
    try:
        return wkt.loads(str(x))
    except:
        return None

df_streets["geom_clean"] = df_streets["geom"].astype(str).str.replace(r"SRID=\d+;", "", regex=True)
df_streets["geometry"] = df_streets["geom_clean"].apply(parse_wkt_safe)
strassen_gdf = gpd.GeoDataFrame(df_streets.dropna(subset=["geometry"]), geometry="geometry", crs="EPSG:4326")

# Straßennamen vereinheitlichen
strassen_gdf["strasse_key"] = strassen_gdf["strasse"].str.strip().str.lower()

# 2. Tätigkeitsfeld-Daten laden (df_all)

df_all["strasse_key"] = df_all["Straßenname"].str.strip().str.lower()

# Merge: Straßen + Tätigkeitsfeld
strassen_merge = strassen_gdf.merge(
    df_all[["strasse_key", "Benennungstyp"]], # Nur die benötigten Spalten aus df_all
    left_on="strasse_key",
    right_on="strasse_key",
    how="left"
)


# 3. Gemarkungen laden (GeoJSON)
gemarkungen_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L73/items"
try:
    stadt_json = requests.get(gemarkungen_url).json()
    stadt_gdf = gpd.GeoDataFrame.from_features(stadt_json["features"])
    stadt_gdf = stadt_gdf.set_crs(epsg=4326)
    print("Gemarkungen erfolgreich geladen.")
except Exception as e:
    print(f"Fehler beim Laden der Gemarkungen: {e}")



# 4. Räumliche Verknüpfung Straße ↔ Gemarkung
# WICHTIG: Sicherstellen, dass beide GeoDataFrames ein gültiges CRS haben
if strassen_gdf.crs != stadt_gdf.crs:
    print("Warnung: CRS stimmen nicht überein. Projiziere strassen_gdf neu...")
    strassen_gdf = strassen_gdf.to_crs(stadt_gdf.crs)

strassen_gemarkung = gpd.sjoin(strassen_merge, stadt_gdf, how="left", predicate="intersects")
print("Räumliche Verknüpfung (sjoin) abgeschlossen.")

# VORBEREITUNG FÜR DIE SCHLEIFE

# Alle einzigartigen Benennungstypen finden
all_types = strassen_gemarkung["Benennungstyp"].dropna().unique()
print(f"Gefundene Benennungstypen zum Iterieren: {all_types}")

# Gesamtanzahl aller nach Personen/Typen benannter Straßen pro Gemarkung (Nenner für die Prozentrechnung)
personen_strassen = strassen_gemarkung.dropna(subset=["Benennungstyp"])
personen_strassen_unique = personen_strassen.drop_duplicates(subset=["gem_nam", "strasse_key"])
gesamt_pro_gemarkung = personen_strassen_unique.groupby("gem_nam").size()

# Kartenmittelpunkt (muss auch nur einmal berechnet werden)
center = stadt_gdf.geometry.centroid.unary_union.centroid

print("Starte Kartenerstellung (alle Straßen in Rot)...")

# START: Iteration durch alle Benennungstypen

for ausgewaehltes_feld in all_types:
    
    print(f"\n--- Verarbeite Kategorie: {ausgewaehltes_feld} ---")

    # 5. Nur Straßen mit *aktuellem* Tätigkeitsfeld filtern
    strassen_feld = strassen_gemarkung[strassen_gemarkung["Benennungstyp"] == ausgewaehltes_feld]
    strassen_feld_unique = strassen_feld.drop_duplicates(subset=["gem_nam", "strasse_key"])

    # 6. Prozentualer Anteil pro Gemarkung berechnen
    feld_counts = strassen_feld_unique.groupby("gem_nam").size()
    anteil_feld = (feld_counts / gesamt_pro_gemarkung).fillna(0).reset_index()
    anteil_feld.columns = ["gem_nam", "anteil_feld"]
    stadt_gdf_feld = stadt_gdf.merge(anteil_feld, on="gem_nam", how="left")
    stadt_gdf_feld["anteil_feld"] = stadt_gdf_feld["anteil_feld"].fillna(0)

    # 7. Karte erstellen (neu für jede Iteration)
    karte = folium.Map(location=[center.y, center.x], zoom_start=12)

    # Choropleth für Gemarkung
    folium.Choropleth(
        geo_data=stadt_gdf_feld,
        data=stadt_gdf_feld,
        columns=["gem_nam", "anteil_feld"],
        key_on="feature.properties.gem_nam",
        fill_color="PuBu",
        fill_opacity=0.6,
        line_opacity=0.2,
        legend_name=f"Anteil '{ausgewaehltes_feld}' Straßen pro Gemarkung"
    ).add_to(karte)

    # Tooltip
    folium.GeoJson(
        stadt_gdf_feld,
        tooltip=folium.GeoJsonTooltip(fields=["gem_nam", "anteil_feld"],
                                        aliases=["Gemarkung", f"Anteil '{ausgewaehltes_feld}' (%)"],
                                        localize=True,
                                        sticky=False),
        style_function=lambda x: {"fillOpacity": 0, "color": "black", "weight": 0.5}
    ).add_to(karte)

    
    # 8. Straßen des Tätigkeitsfelds einzeichnen (VEREINFACHT)

    # Farbe hart auf Rot setzen
    farbe_strassen = "red"

    for _, row in strassen_feld.iterrows(): # strassen_feld (nicht _unique) für alle Geometrien
        geom = row['geometry']
        if geom.geom_type == "LineString":
            punkte = [(pt[1], pt[0]) for pt in geom.coords]
            folium.PolyLine(punkte, color=farbe_strassen, weight=2, opacity=0.7).add_to(karte)
        elif geom.geom_type == "MultiLineString":
            for line in geom.geoms:
                punkte = [(pt[1], pt[0]) for pt in line.coords]
                folium.PolyLine(punkte, color=farbe_strassen, weight=2, opacity=0.7).add_to(karte)

    # 9. Karte speichern & anzeigen
    
    # Dateinamen "säubern" (z.B. '/' durch '_' ersetzen)
    safe_filename = re.sub(r'[^\w\-_\. ]', '_', ausgewaehltes_feld)
    filename = f"../../results/maps/Gemarkungen/Karte_Gemarkung_Benennungstyp_{safe_filename}.html"
    
    karte.save(filename)

Gemarkungen erfolgreich geladen.
Räumliche Verknüpfung (sjoin) abgeschlossen.
Gefundene Benennungstypen zum Iterieren: ['Person real' 'Ort' 'Region' 'Bevölkerungsgruppe' 'Bauwerk/Infrastruktur'
 'Sonstige' 'Selbstreferenziell' 'Natur' 'Gegenstand' 'weitere Geografika'
 'Gewerbe/Industrie' 'Gruppe real' 'Beruf/Funktion' 'Abstrakta' 'Ereignis'
 'Person fiktiv' 'Unbekannt' 'Siedlung' 'Burg' 'Land' 'Kunstwerk']


C:\Users\juliu\AppData\Local\Temp\ipykernel_22556\2297245745.py:65: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = stadt_gdf.geometry.centroid.unary_union.centroid
C:\Users\juliu\AppData\Local\Temp\ipykernel_22556\2297245745.py:65: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = stadt_gdf.geometry.centroid.unary_union.centroid


Starte Kartenerstellung (alle Straßen in Rot)...

--- Verarbeite Kategorie: Person real ---

--- Verarbeite Kategorie: Ort ---

--- Verarbeite Kategorie: Region ---

--- Verarbeite Kategorie: Bevölkerungsgruppe ---

--- Verarbeite Kategorie: Bauwerk/Infrastruktur ---

--- Verarbeite Kategorie: Sonstige ---

--- Verarbeite Kategorie: Selbstreferenziell ---

--- Verarbeite Kategorie: Natur ---

--- Verarbeite Kategorie: Gegenstand ---

--- Verarbeite Kategorie: weitere Geografika ---

--- Verarbeite Kategorie: Gewerbe/Industrie ---

--- Verarbeite Kategorie: Gruppe real ---

--- Verarbeite Kategorie: Beruf/Funktion ---

--- Verarbeite Kategorie: Abstrakta ---

--- Verarbeite Kategorie: Ereignis ---

--- Verarbeite Kategorie: Person fiktiv ---

--- Verarbeite Kategorie: Unbekannt ---

--- Verarbeite Kategorie: Siedlung ---

--- Verarbeite Kategorie: Burg ---

--- Verarbeite Kategorie: Land ---

--- Verarbeite Kategorie: Kunstwerk ---

--- Alle Karten erfolgreich erstellt. ---


### Statisches PNG

In [14]:
# 1. Lokale Straßendaten laden (df_streets)

# WKT-Geometrien bereinigen und umwandeln
def parse_wkt_safe(x):
    if pd.isna(x):
        return None
    try:
        return wkt.loads(str(x))
    except:
        return None

df_streets["geom_clean"] = df_streets["geom"].astype(str).str.replace(r"SRID=\d+;", "", regex=True)
df_streets["geometry"] = df_streets["geom_clean"].apply(parse_wkt_safe)
strassen_gdf = gpd.GeoDataFrame(df_streets.dropna(subset=["geometry"]), geometry="geometry", crs="EPSG:4326")

# Straßennamen vereinheitlichen
strassen_gdf["strasse_key"] = strassen_gdf["strasse"].str.strip().str.lower()

# 2. Kategorie-Daten laden (df_all)

df_all["strasse_key"] = df_all["Straßenname"].str.strip().str.lower()

# Merge: Straßen + Kategorie
strassen_merge = strassen_gdf.merge(
    df_all[["strasse_key", "Benennungstyp"]], # Nur die benötigten Spalten aus df_all
    left_on="strasse_key",
    right_on="strasse_key",
    how="left"
)


# 3. Gemarkungen laden (GeoJSON)
gemarkungen_url = "https://kommisdd.dresden.de/net4/public/ogcapi/collections/L73/items"
try:
    stadt_json = requests.get(gemarkungen_url).json()
    stadt_gdf = gpd.GeoDataFrame.from_features(stadt_json["features"])
    stadt_gdf = stadt_gdf.set_crs(epsg=4326)
except Exception as e:
    print(f"Fehler beim Laden der Gemarkungen: {e}")
    # Hier ggf. Skript beenden oder mit lokalen Daten weitermachen

# 4. Räumliche Verknüpfung Straße ↔ Gemarkung
if strassen_gdf.crs != stadt_gdf.crs:
    print("Warnung: CRS stimmen nicht überein. Projiziere strassen_gdf neu...")
    strassen_gdf = strassen_gdf.to_crs(stadt_gdf.crs)

strassen_gemarkung = gpd.sjoin(strassen_merge, stadt_gdf, how="left", predicate="intersects")

# VORBEREITUNG FÜR DIE SCHLEIFE

# Alle einzigartigen Benennungstypen finden
all_types = strassen_gemarkung["Benennungstyp"].dropna().unique()

# Gesamtanzahl aller nach Personen/Typen benannter Straßen pro Bezirk (Nenner für die Prozentrechnung)
personen_strassen = strassen_gemarkung.dropna(subset=["Benennungstyp"])
personen_strassen_unique = personen_strassen.drop_duplicates(subset=["gem_nam", "strasse_key"])
gesamt_pro_gemarkung = personen_strassen_unique.groupby("gem_nam").size()

# Kartenmittelpunkt
center = stadt_gdf.geometry.centroid.unary_union.centroid


print("Starte Erstellung von statischen PNG-Karten...")

# START: Iteration durch alle Benennungstypen
for ausgewaehltes_feld in all_types:
    
    print(f"\n--- Verarbeite Kategorie: {ausgewaehltes_feld} ---")

    # 5. Filtern
    strassen_feld = strassen_gemarkung[strassen_gemarkung["Benennungstyp"] == ausgewaehltes_feld]

    # 6. Prozentrechnung
    strassen_feld_unique = strassen_feld.drop_duplicates(subset=["gem_nam", "strasse_key"])
    feld_counts = strassen_feld_unique.groupby("gem_nam").size()
    anteil_feld = (feld_counts / gesamt_pro_gemarkung).fillna(0).reset_index()
    anteil_feld.columns = ["gem_nam", "anteil_feld"]
    stadt_gdf_feld = stadt_gdf.merge(anteil_feld, on="gem_nam", how="left")
    stadt_gdf_feld["anteil_feld"] = stadt_gdf_feld["anteil_feld"].fillna(0)

    # Spalte für die Legende (Werte von 0-100)
    stadt_gdf_feld["anteil_prozent"] = stadt_gdf_feld["anteil_feld"] * 100

    
    # 7. Daten für Plotting vorbereiten (CRS umwandeln)
    #    Basemaps benötigen EPSG:3857 (Web Mercator)
    target_crs = "EPSG:3857"
    stadt_plot = stadt_gdf_feld.to_crs(target_crs)
    strassen_plot = strassen_feld.to_crs(target_crs)

    # 8. Karte erstellen
    fig, ax = plt.subplots(figsize=(12, 12)) # 12x12 Zoll für gute Auflösung

    # A: Gemarkungen als Choropleth plotten
    stadt_plot.plot(
        ax=ax,
        column="anteil_prozent", # Spalte für die Farbe
        cmap="PuBu",          # Farbschema
        legend=True,          # Zeigt die Legende
        alpha=0.6,
        edgecolor='black',      
        linewidth=0.5,                    
        legend_kwds={
            'label': f"Anteil '{ausgewaehltes_feld}' (%)",
            'shrink': 0.75, 
            'aspect': 30   
        }
    )
    
    # Legenden-Schriftgröße anpassen
    cax = fig.axes[-1] 
    cax.tick_params(labelsize=8) 
    cax.yaxis.label.set_size(9)  

    # B: Straßen-Linien plotten
    strassen_plot.plot(
        ax=ax,
        color="red",
        linewidth=0.7,      
        alpha=0.5           
    )

    # C: Hintergrundkarte hinzufügen
    ctx.add_basemap(
        ax,
        crs=target_crs,
        source=ctx.providers.OpenStreetMap.Mapnik,
        zoom=13
    )

    # 9. Karte finalisieren & als PNG speichern
    ax.set_axis_off() 
    plt.title(f"Verteilung Straßennamen: {ausgewaehltes_feld}", fontsize=16)
    
    # Dateinamen "säubern"
    safe_filename = re.sub(r'[^\w\-_\. ]', '_', ausgewaehltes_feld)
    
    # Ordner für PNGs
    filename = f"../../results/images/Gemarkungen/Karte_Gemarkung_Benennungstyp_{safe_filename}.png"
    
    plt.savefig(
        filename,
        dpi=400, 
        bbox_inches="tight" 
    )
    plt.close(fig) 

print("\n--- Alle statischen PNG-Karten erfolgreich erstellt. ---")

C:\Users\juliu\AppData\Local\Temp\ipykernel_19916\3216488217.py:60: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = stadt_gdf.geometry.centroid.unary_union.centroid
C:\Users\juliu\AppData\Local\Temp\ipykernel_19916\3216488217.py:60: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = stadt_gdf.geometry.centroid.unary_union.centroid


Starte Erstellung von statischen PNG-Karten...

--- Verarbeite Kategorie: Person real ---

--- Verarbeite Kategorie: Ort ---

--- Verarbeite Kategorie: Region ---

--- Verarbeite Kategorie: Bevölkerungsgruppe ---

--- Verarbeite Kategorie: Bauwerk/Infrastruktur ---

--- Verarbeite Kategorie: Sonstige ---

--- Verarbeite Kategorie: Selbstreferenziell ---

--- Verarbeite Kategorie: Natur ---

--- Verarbeite Kategorie: Gegenstand ---

--- Verarbeite Kategorie: weitere Geografika ---

--- Verarbeite Kategorie: Gewerbe/Industrie ---

--- Verarbeite Kategorie: Gruppe real ---

--- Verarbeite Kategorie: Beruf/Funktion ---

--- Verarbeite Kategorie: Abstrakta ---

--- Verarbeite Kategorie: Ereignis ---

--- Verarbeite Kategorie: Person fiktiv ---

--- Verarbeite Kategorie: Unbekannt ---

--- Verarbeite Kategorie: Siedlung ---

--- Verarbeite Kategorie: Burg ---

--- Verarbeite Kategorie: Land ---

--- Verarbeite Kategorie: Kunstwerk ---

--- Alle statischen PNG-Karten erfolgreich erstellt. ---